In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import pickle
import sqlite3
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def preprocess_data(file_path='traffic_volume_data.csv'):
    """
    Preprocess the traffic volume data and store in SQLite database
    """
    # Load data
    data = pd.read_csv(file_path)

    # Sort by date and time
    data = data.sort_values(by=['date_time'], ascending=True).reset_index(drop=True)

    # Create lag features for the last n hours
    last_n_hours = [1, 2, 3, 4, 5, 6]
    for n in last_n_hours:
        data[f'last_{n}_hour_traffic'] = data['traffic_volume'].shift(n)

    # Drop rows with NaN values
    data = data.dropna().reset_index(drop=True)

    # Convert holiday to binary
    data.loc[data['is_holiday'] != 'None', 'is_holiday'] = 1
    data.loc[data['is_holiday'] == 'None', 'is_holiday'] = 0
    data['is_holiday'] = data['is_holiday'].astype(int)

    # Extract datetime features
    data['date_time'] = pd.to_datetime(data['date_time'])
    data['hour'] = data['date_time'].map(lambda x: int(x.strftime("%H")))
    data['month_day'] = data['date_time'].map(lambda x: int(x.strftime("%d")))
    data['weekday'] = data['date_time'].map(lambda x: x.weekday()+1)
    data['month'] = data['date_time'].map(lambda x: int(x.strftime("%m")))
    data['year'] = data['date_time'].map(lambda x: int(x.strftime("%Y")))

    # Convert weather types and descriptions to numerical values
    weather_type_mapping = {
        'Rain': 1, 'Clouds': 2, 'Clear': 3, 'Snow': 4, 'Mist': 5,
        'Drizzle': 6, 'Haze': 7, 'Thunderstorm': 8, 'Fog': 9, 'Smoke': 10, 'Squall': 11
    }

    weather_desc_mapping = {
        'SQUALLS': 1, 'Sky is Clear': 2, 'broken clouds': 3, 'drizzle': 4,
        'few clouds': 5, 'fog': 6, 'freezing rain': 7, 'haze': 8,
        'heavy intensity drizzle': 9, 'heavy intensity rain': 10, 'heavy snow': 11,
        'light intensity drizzle': 12, 'light intensity shower rain': 13, 'light rain': 14,
        'light rain and snow': 15, 'light shower snow': 16, 'light snow': 17,
        'mist': 18, 'moderate rain': 19, 'overcast clouds': 20,
        'proximity shower rain': 21, 'proximity thunderstorm': 22,
        'proximity thunderstorm with drizzle': 23, 'proximity thunderstorm with rain': 24,
        'scattered clouds': 25, 'shower snow': 26, 'sky is clear': 27,
        'sleet': 28, 'smoke': 29, 'snow': 30, 'thunderstorm': 31,
        'thunderstorm with drizzle': 32, 'thunderstorm with heavy rain': 33,
        'thunderstorm with light drizzle': 34, 'thunderstorm with light rain': 35,
        'thunderstorm with rain': 36, 'very heavy rain': 37
    }

    # Apply mappings with a default of 0 for any value not in the mapping
    data['weather_type_num'] = data['weather_type'].map(
        lambda x: weather_type_mapping.get(x, 0))
    data['weather_description_num'] = data['weather_description'].map(
        lambda x: weather_desc_mapping.get(x, 0))

    # Save preprocessed data to CSV
    data.to_csv("cleaned_data.csv", index=None)

    # Return processed data
    return data

In [3]:
def create_lstm_model(input_shape):
    """Create and compile LSTM model"""
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [4]:
def prepare_lstm_data(data, lookback=6, feature_cols=None, target_col='traffic_volume'):
    """
    Prepare data for LSTM model with lookback sequence
    """
    if feature_cols is None:
        feature_cols = ['is_holiday', 'humidity', 'wind_speed', 'temperature',
                        'weekday', 'hour', 'month_day', 'year', 'month']

    # Extract features and target
    X = data[feature_cols].values
    y = data[target_col].values

    # Scale features
    x_scaler = MinMaxScaler()
    X = x_scaler.fit_transform(X)

    # Scale target
    y_scaler = MinMaxScaler()
    y = y_scaler.fit_transform(y.reshape(-1, 1)).flatten()

    # Save scalers for later use in prediction
    with open('x_scaler.pkl', 'wb') as file:
        pickle.dump(x_scaler, file)

    with open('y_scaler.pkl', 'wb') as file:
        pickle.dump(y_scaler, file)

    # Create sequences for LSTM
    X_seq, y_seq = [], []
    for i in range(len(X) - lookback):
        X_seq.append(X[i:i+lookback])
        y_seq.append(y[i+lookback])

    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)

    return X_seq, y_seq, x_scaler, y_scaler

In [5]:
def train_model(data, epochs=50, batch_size=32, validation_split=0.2):
    """Train LSTM model and save it"""
    # Define feature columns
    feature_cols = ['is_holiday', 'humidity', 'wind_speed', 'temperature',
                    'weekday', 'hour', 'month_day', 'year', 'month']

    # Prepare data for LSTM
    X_seq, y_seq, x_scaler, y_scaler = prepare_lstm_data(data, lookback=6,
                                                       feature_cols=feature_cols)

    # Split into train and validation sets
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(X_seq, y_seq,
                                                      test_size=validation_split,
                                                      random_state=42)

    # Create and train model
    model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Save the trained model
    model.save('traffic_lstm_model.h5')
    print("Model trained and saved successfully!")

    # Evaluate model
    train_loss, train_mae = model.evaluate(X_train, y_train, verbose=0)
    val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)

    print(f"Training MAE: {train_mae:.4f}")
    print(f"Validation MAE: {val_mae:.4f}")

In [8]:
if __name__ == "__main__":
    # Process data
    data = preprocess_data('/content/drive/MyDrive/traffic_prediction_6/traffic_volume_data.csv')

    # Train and save model
    train_model(data, epochs=50)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


844/844 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.0719 - mae: 0.2088 - val_loss: 0.0289 - val_mae: 0.1266
Epoch 2/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0280 - mae: 0.1222 - val_loss: 0.0240 - val_mae: 0.1106
Epoch 3/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 0.0249 - mae: 0.1145 - val_loss: 0.0219 - val_mae: 0.1044
Epoch 4/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.0220 - mae: 0.1078 - val_loss: 0.0178 - val_mae: 0.0917
Epoch 5/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - loss: 0.0185 - mae: 0.0981 - val_loss: 0.0157 - val_mae: 0.0866
Epoch 6/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - loss: 0.0157 - mae: 0.0891 - val_loss: 0.0154 - val_mae: 0.0824
Epoch 7/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/step - loss: 0.0137 - mae: 0.0813 - val_loss: 0.0127 - val_mae: 0.0776
Epoch 8/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0130 - mae: 0.0785 - val_loss: 0.0116 - val_mae: 0.0750
Epoch 9/50
844/844 ━━━━━━━━━━━━━━━━━━━━ 19s 10ms/ste

Model trained and saved successfully!
Training MAE: 0.0564
Validation MAE: 0.0590


In [11]:
model = load_model('/content/traffic_lstm_model.h5', compile=False)


In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import json

def save_model_components():
    """
    Save model architecture and weights separately
    to handle loading issues with custom metrics
    """
    try:
        # Load model without compiling to avoid metric deserialization issues
        model = load_model('/content/traffic_lstm_model.h5', compile=False)

        # Save model architecture to JSON
        model_json = model.to_json()
        with open('/content/drive/MyDrive/traffic_prediction_6/model_architecture.json', 'w') as json_file:
            json_file.write(model_json)

        # Save weights separately
        model.save_weights('/content/drive/MyDrive/traffic_prediction_6/model_weights.weights.h5')

        print("Model architecture and weights saved successfully!")
        return True
    except Exception as e:
        print(f"Error saving model components: {str(e)}")
        return False

if __name__ == "__main__":
    save_model_components()


Model architecture and weights saved successfully!
